# Docker Lab Contents

The goal of this lab is to install and use Docker to become familiar with Linux based containers and handle some of the common use cases around it. By the end of this lab you will have created a Web application comprised of a number of micro-services.

## Lab Writers and Trainers
  - Bruno.Cornec@hpe.com
  - Rene.Ribaud@hpe.com

Table of Contents
=================

   * [Docker Lab Contents](#docker-lab-contents)
      * [Lab Writers and Trainers](#lab-writers-and-trainers)
      * [Objectives of the Docker Lab](#objectives-of-the-docker-lab)
      * [Reference documents](#reference-documents)
      * [Note on Linux commands](#note-on-linux-commands)
   * [Environment setup](#environment-setup)
      * [Proxy consideration](#proxy-consideration)
      * [Docker installation](#docker-installation)
         * [Ubuntu installation](#ubuntu-installation)
         * [CentOS installation](#centos-installation)
         * [Check installation](#check-installation)
   * [Using Docker](#using-docker)
      * [The first container](#the-first-container)
      * [The second container](#the-second-container)
   * [Configuring owncloud in a container](#configuring-owncloud-in-a-container)
   * [Using Docker compose](#using-docker-compose)
      * [Installing Docker compose](#installing-docker-compose)
      * [Our first docker-compose.yml file](#our-first-docker-composeyml-file)
      * [Going further with docker-compose.yml](#going-further-with-docker-composeyml)
   * [Using docker-machine to create Docker hosts](#using-docker-machine-to-create-docker-hosts)
   * [Using Docker Swarm](#using-docker-swarm)
      * [Installing Docker Swarm](#installing-docker-swarm)
      * [Installing on CentOS 7](#installing-on-centos-7)
      * [Installing the engine in the Cloud](#installing-the-engine-in-the-cloud)
      * [Using Docker Swarm to make our configuration available and scalable](#using-docker-swarm-to-make-our-configuration-available-and-scalable)
         * [CentOS 7](#centos-7)
         * [Ubuntu](#ubuntu)
   * [Deploy a cloud native application.](#deploy-a-cloud-native-application)
      * [Objectives](#objectives)

## Objectives of the Docker Lab
At the end of the Lab students should be able to install Docker, use the CLI to create a new image, a container, launch an application in it, store data, configure the network.

This Lab is intended to be trial and error so that during the session students should understand really what is behind the tool.  Blindly following instructions is not an effective way to learn IMHO. You've been warned ;-)

Expected duration : 120 minutes

## Reference documents
When dealing with the installation and configuration of Docker, the first step  is to check the reference Web site http://docker.io/:

At the start of each section there is an estimate of how long it will take to complete.

## Note on Linux commands

If you are familiar with Linux, you can skip this section. If not please read to understand some commands.

In a number of places, the lab uses the Linux here pattern to create text files, as in the following example,

`#` **`cat > fileToCreate << EOF`**
```none
Text line 1
Text line 2
EOF
```

This command will create the text file `fileToCreate` and populate it with the lines of text that follow up but not including the EOF keyword.

You can display the content of the created file with the command `cat fileToCreate`.

In order to append text to the file, the first `>` can be replaced with `>>`.

If you prefer, you can edit the files using **vim** or **nano** text editors.

## But I'm a poor lonesome Windows cowboy !

Well in that case, first condolences, then read the fine document made by Michael Mayer at https://github.com/bcornec/Labs/blob/master/Docker/LabOnWindows.md to get help.

# Environment setup
Estimated time: 15 minutes

## Lab setup

Please refer to the instructions available at https://github.com/bcornec/Labs/blob/master/ENVIRONMENT.md

Note that Docker is also providing the possibility to have a web based access to an on demand infrastructure for 4 hours. This is available at http://play-with-docker.com. Check that as a fallback you can use it and create up to 5 nodes with it. Become familiar to use this or your platform of choice for the rest of the Lab.

## Proxy consideration

This lab is usually run in our environment that has a direct access to the Internet. If you want to run this lab on your site behind a corporate proxy, you will have to configure your Linux distribution and Docker to access the Internet via your proxy.

The following instruction are available for a Centos 7 distribution, because instructions on the Docker part are systemd based.


 1. Get the proxy IP and port.
 2. Make sure your host can resolve the proxy address using `nslookup <proxy>`, if not use the proxy IP.
 3. Configure your Linux package manager to go through the proxy by exporting the http_proxy and https_proxy environment variables:

```
export http_proxy=http://<proxy name or ip>:<proxy port>
export https_proxy=http://<proxy name or ip>:<proxy port>
```

 4. Configure Docker daemon to use the proxy as explained by this document: https://docs.docker.com/engine/admin/systemd/#http-proxy (in short add Environment="HTTP_PROXY=http://proxy.example.com:80/" "HTTPS_PROXY=http://proxy.example.com:80/" to your [services] section)
<!-- Previous version
 5. **Set the proxy in each of your Dockerfiles** by adding following text as the 2nd and 3rd line of the Dockerfile.

```
ENV http_proxy <HTTP_PROXY>
ENV https_proxy <HTTP_PROXY>
```
-->
 5. Build Dockerfile with the appropriate arguments.

```
docker build --build-arg http_proxy=http://<proxy name or ip>:<proxy port> https_proxy=http://<proxy name or ip>:<proxy port> .
```

Documentation details :
https://docs.docker.com/engine/reference/builder/#arg

## Docker installation
Docker is available externally from http://docs.docker.com/linux/step_one/ or using your distribution packages, or from github at https://github.com/docker/docker
Version 19.03 is the current stable release. This lab requires at least version 1.7.

Ask to your instructor which Linux distribution will be used for the Lab (Docker in Docker, Ubuntu or CentOS). Then refer to the corresponding instructions below.

Other distributions should be as easy to deal with once the same packages have been installed using the package manager as they should be available directly (Case of most non-commercial distributions such as Debian, Fedora, Mageia, OpenSUSE, ...). Follow the instructions from https://docs.docker.com/engine/installation/

### Docker in Docker installation

As the docker image already provides the required packages, you just need to check that docker is running and at the correct minimal version. Go to the Check installation paragrah below for that.

### Ubuntu installation
If you work on an Ubuntu environment for the Lab, you may want to use apt to do the installation of Docker with all its dependencies. As Ubuntu provides an old version of Docker, we will use a PPA providing a more up to date version:

#### 16.04 and later

`#` **`sudo apt-get update`**

`#` **`sudo apt-get install apt-transport-https ca-certificates curl software-properties-common`**

`#` **`curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -`**

`#` **`sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"`**

`#` **`sudo apt-get update`**

`#` **`sudo apt-get install docker-ce`**

#### 14.04

`#` **`apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 36A1D7869245C8950F966E92D8576A8BA88D21E9`**

`#` **`echo deb https://get.docker.io/ubuntu docker main > /etc/apt/sources.list.d/Docker.list`**

`#` **`apt-get update`**

`#` **`apt-get install lxc-docker`**
```
Reading package lists... Done
Building dependency tree
Reading state information... Done
The following extra packages will be installed:
  aufs-tools cgroup-lite git git-man liberror-perl patch
Suggested packages:
  btrfs-tools debootstrap lxc rinse git-daemon-run git-daemon-sysvinit git-doc
  git-el git-email git-gui gitk gitweb git-arch git-bzr git-cvs git-mediawiki
  git-svn diffutils-doc
The following NEW packages will be installed:
  aufs-tools cgroup-lite lxc-docker git git-man liberror-perl patch
0 upgraded, 7 newly installed, 0 to remove and 0 not upgraded.
Need to get 7,640 kB of archives.
After this operation, 46.9 MB of additional disk space will be used.
Do you want to continue? [Y/n] y
Get:1 http://fr.archive.ubuntu.com/ubuntu/ trusty/universe aufs-tools amd64 1:3.2+20130722-1.1 [92.3 kB]
Get:2 https://get.docker.io/ubuntu/ docker/main lxc-docker-1.7.0 amd64 1.7.0 [4,962 kB]
[...]
Fetched 7,640 kB in 8s (884 kB/s)
Selecting previously unselected package aufs-tools.
(Reading database ... 54255 files and directories currently installed.)
Preparing to unpack .../aufs-tools_1%3a3.2+20130722-1.1_amd64.deb ...
Unpacking aufs-tools (1:3.2+20130722-1.1) ...
[...]
Setting up lxc-docker (1.7.0) ...
Adding group docker' (GID 111) ...
Done.
[...]
```

### Debian installation

Docker is providing deb packages to help you install the Engine on your Debian distribution:

`#` **`wget -O- https://apt.dockerproject.org/gpg | apt-key add -`**

`#` **`echo deb https://apt.dockerproject.org/repo debian-stretch main > /etc/apt/sources.list.d/Docker.list`**

`#` **`apt-get update`**

`#` **`apt-get install docker-engine`**

This procedure should also work for Ubuntu based distributions.

### CentOS installation

If you work on a CentOS 7 environment for the Lab, you may want to use yum to do the installation of Docker with all its dependencies. Add the repo provided by the Docker project (which is requiring 7.2 at least, but not by CentOS if you use that variant):

`#` **`cat > /etc/yum.repos.d/docker.repo << EOF`**
```none
[dockerrepo]
name=Docker Repository
baseurl=https://yum.dockerproject.org/repo/main/centos/7
enabled=1
gpgcheck=1
gpgkey=https://yum.dockerproject.org/gpg
EOF
```

`#` **`yum install docker-engine`**
```
Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: mirror.denit.net
 * extras: centos.mirror.triple-it.nl
 * updates: mirrors.supportex.net
base                                                                              | 3.6 kB  00:00:00
dockerrepo                                                                        | 2.9 kB  00:00:00
extras                                                                            | 3.4 kB  00:00:00
updates                                                                           | 3.4 kB  00:00:00
(1/5): base/7/x86_64/group_gz                                                     | 166 kB  00:00:00
(2/5): extras/7/x86_64/primary_db                                                 | 149 kB  00:00:00
(3/5): dockerrepo/primary_db                                                      |  34 kB  00:00:00
(4/5): updates/7/x86_64/primary_db                                                | 2.7 MB  00:00:00
(5/5): base/7/x86_64/primary_db                                                   | 5.9 MB  00:00:00
Resolving Dependencies
--> Running transaction check
---> Package docker-engine.x86_64 0:18.06.0.ce-1.el7.centos will be installed
--> Processing Dependency: docker-engine-selinux >= 18.06.0.ce-1.el7.centos for package: docker-engine-18.06.0.ce-1.el7.centos.x86_64
--> Processing Dependency: libltdl.so.7()(64bit) for package: docker-engine-18.06.0.ce-1.el7.centos.x86_64
--> Running transaction check
---> Package docker-engine-selinux.noarch 0:18.06.0.ce-1.el7.centos will be installed
---> Package libtool-ltdl.x86_64 0:2.4.2-22.el7_3 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

=========================================================================================================
 Package                       Arch           Version                           Repository          Size
=========================================================================================================
Installing:
 docker-engine                 x86_64         18.06.0.ce-1.el7.centos           dockerrepo          19 M
Installing for dependencies:
 docker-engine-selinux         noarch         18.06.0.ce-1.el7.centos           dockerrepo          28 k
 libtool-ltdl                  x86_64         2.4.2-22.el7_3                    base                49 k

Transaction Summary
=========================================================================================================
Install  1 Package (+2 Dependent packages)

Total download size: 20 M
Installed size: 70 M
Is this ok [y/d/N]: y
Downloading packages:.
[...]
```

`#` **`systemctl start docker`**

### Windows installation

Look at these contributed instructions instead: https://github.com/bcornec/Labs/blob/master/Docker/LabOnWindows.md

### MacOS installation

Download the Docker for MacOS version on https://download.docker.com/mac/stable/Docker.dmg
Then create a subdirectory under you home directory and do all the steps below in it.

### Check installation

If you're not using the Docker in Docker environement:

`#` **`systemctl status docker`**

Now for all versions, check that the correct version is installed and operational:

`#` **`docker --version`**
```
Docker version 18.05.0-ce, build f150324
```
Note : The version could be different on your system.

`#` **`docker info`**
```
Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: 18.06.1-ce
Storage Driver: devicemapper
 Pool Name: docker-253:2-130978-pool
 Pool Blocksize: 65.54 kB
 Base Device Size: 10.74 GB
 Backing Filesystem: xfs
 Data file: /dev/loop0
 Metadata file: /dev/loop1
 Data Space Used: 11.8 MB
 Data Space Total: 107.4 GB
[...]
Cgroup Driver: cgroupfs
Plugins:
 Volume: local
 Network: null host bridge
Kernel Version: 3.10.0-327.el7.x86_64
Operating System: Red Hat Enterprise Linux Server 7.2 (Maipo)
OSType: linux
Architecture: x86_64
CPUs: 6
Total Memory: 15.39 GiB
Name: lab3.labossi.hpintelco.org
ID: JFU6:LTUL:UOB2:4NEE:IZFC:FZK7:INUC:7ABM:JRVG:NQOS:VSXH:4XMG
Docker Root Dir: /var/lib/docker
Debug Mode (client): false
Debug Mode (server): false
Registry: https://index.docker.io/v1/
WARNING: bridge-nf-call-iptables is disabled
WARNING: bridge-nf-call-ip6tables is disabled
```

`#` **`docker `**

[Display online help]

Now that the software has been installed, we'll use it to create and manage containers.




# Using Docker Swarm

Docker Swarm is, since version 1.12, part of Docker Engine.
It is used to provide high availability for Docker containers.

A really complete and excellent workshop is available for Swarm at https://jpetazzo.github.io/orchestration-workshop
We extracted lots of ideas from it to lead you towards a first understanding of Swarm.

We will deploy a 5 nodes (3 X master + 2 X workers) cluster.

Note : If you are late on this lab, you can just use 1 X master and 2 workers, but do not stop the master in further steps.

## Installing Docker Swarm

If you have a version prior to 1.13, then you'll need to install Docker Engine 1.13+ as the rest of this lab requires that version.

## Installing on CentOS 7

On CentOS 7 just add the repo file mentioned earlier in this Lab to get it.

<!--
## Installing on Ubuntu
## Installing the engine manually
-->

## Installing the engine in the Cloud

If you followed docker-machine part, you can now use these machines to configure a Swarm cluster as you have the latest version available in them.

## Using Docker Swarm to make our configuration available and scalable

So now that we can orchestrate the creation of our 2 containers hosting our
application, we would like to make it scalable and error proof. Let's try to
look at which nodes are available on our cluster:

`#` **`docker node ls`**
```
Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
```
Ok, so you need first to initiate a swarm cluster ! Let's do it on our node as
instructed:

`#` **`docker swarm init`**
```
Swarm initialized: current node (82mm398yajdl4lor2gzc4eeeo) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-444fdgnkvchgol08ck8rexwhxg8hbvwncyqs61mvcu0b3978qs-4r5p96yudo5r1x6c4psxd1uyt \
    10.11.51.136:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.
```

So use the previous advise to add your other nodes to the Swarm cluster as worker.

`#` **`docker node ls`**
```
ID                           HOSTNAME  STATUS  AVAILABILITY  MANAGER STATUS
2cosbse8y5o1sl2zr4o2tc06q    c11.labossi.hpintelco.org  Ready   Active
31n32lc4wjv9oskc6ejyvxz9j *  c6.labossi.hpintelco.org   Ready   Active        Leader
51kz6qmid4blq7pjbrq5527o5    c7.labossi.hpintelco.org   Ready   Active
726rg84phfkohofjpn8p2ztfg    c8.labossi.hpintelco.org   Ready   Active
d8dfb2e8qd3h703pw43o5r88f    c10.labossi.hpintelco.org  Ready   Active
```

Check what you can see on each node. Also look at the output of the `docker info` command.

If you have problems with error messages like "Error response from daemon: Timeout was reached before node was joined." then you firewall may be blocking the ports that Docker Swarm uses. If you think this is the case, you may have firewalling issues ;-)

### Configuring firewall on CentOS7

Look at https://www.digitalocean.com/community/tutorials/how-to-configure-the-linux-firewall-for-docker-swarm-on-centos-7

I recommend that you run the following commands on **all** nodes to avoid firewalling issue in the rest of the Lab:

`#` **`firewall-cmd --add-port=2376/tcp --permanent`**

`#` **`firewall-cmd --add-port=2377/tcp --permanent`**

`#` **`firewall-cmd --add-port=7946/tcp --permanent`**

`#` **`firewall-cmd --add-port=7946/udp --permanent`**

`#` **`firewall-cmd --add-port=4789/udp --permanent`**

But you will probably have many issues with firewalld later on anyway, so it's worth disabling it now on all nodes to avoid solving unrelated issues and integration aspects with Docker iptables management (been there done that for hours !). And believe me, I don't like that :-( (so in our Lab we have peripheral firewall !). For that use:

`#` **`systemctl stop firewalld`**
`#` **`systemctl restart docker`**

This previous command re-establish the NAT rules set up by Docker and that
have been reset just previously.

### Configuring firewall on Ubuntu

I recommend that you run the following command on **all** nodes to avoid firewalling issue in the rest of the Lab:

`#` **`ufw disable`**

Back to out normal program now !

Swarm has the notion of worker (hosting containers), manager (able to be also
a worker and being a backup leader) and Leader (manager being in charge of the
Swarm cluster).

In order to render our cluster highly available, we need to have an odd number
of managers. Here we can promote 2 of our workers as managers. For that, we
need to get another token, the manager one, instead of the worker one we used
previously.

Note : If you deployed only 3 nodes, you can not add managers, so skip this part.

`#` **`docker swarm join-token -q manager`**
```
SWMTKN-1-444fdgnkvchgol08ck8rexwhxg8hbvwncyqs61mvcu0b3978qs-cw10maud95375a2t35p7m5kox
```

So now you have the right token, use it as previously on 2 of your nodes to
promote them as managers.

Example:
`#` **`docker node promote c7.labossi.hpintelco.org`**

At the end you should get the following result:

`#` **`docker node ls`**
```
ID                           HOSTNAME  STATUS  AVAILABILITY  MANAGER STATUS
2cosbse8y5o1sl2zr4o2tc06q    c11.labossi.hpintelco.org  Ready   Active
31n32lc4wjv9oskc6ejyvxz9j *  c6.labossi.hpintelco.org   Ready   Active        Leader
51kz6qmid4blq7pjbrq5527o5    c7.labossi.hpintelco.org   Ready   Active        Reachable
726rg84phfkohofjpn8p2ztfg    c8.labossi.hpintelco.org   Ready   Active        Reachable
d8dfb2e8qd3h703pw43o5r88f    c10.labossi.hpintelco.org  Ready   Active
```

There are many ways to do it, including using docker node update.

So now that we have a cluster running, it would be a good idea to launch
containers on it. But in a Swarm cluster this means creating services. So
let's create a simple service to test our cluster:

`#` **`docker service create alpine ping 8.8.8.8`**
```
v12wk2jruwhltftgv5xalaped
overall progress: 1 out of 1 tasks 
1/1: running   [==================================================>] 
verify: Service converged 
```

`#` **`docker service ls`**
```
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
v12wk2jruwhl        relaxed_morse       replicated          1/1                 alpine:latest       
```

`#` **`docker service ps v12`**
```
ID                         NAME             IMAGE          NODE                      DESIRED STATE  CURRENT STATE              ERROR               PORTS
9aq9iq25ayhp1nk11ems7tsly  relaxed_morse.1  alpine:latest  c6.labossi.hpintelco.org  Running        Running 35 seconds ago
```

Use the Docker commands to check how the container is behaving in your
environment. Restart the Docker daemon on the leader node and look at the
cluster behaviour.

You can scale that service:

`#` **`docker service update v12 --replicas 10`**
```
v12
overall progress: 10 out of 10 tasks 
1/10: running   [==================================================>] 
2/10: running   [==================================================>] 
3/10: running   [==================================================>] 
4/10: running   [==================================================>] 
5/10: running   [==================================================>] 
6/10: running   [==================================================>] 
7/10: running   [==================================================>] 
8/10: running   [==================================================>] 
9/10: running   [==================================================>] 
10/10: running   [==================================================>] 
verify: Service converged 
```

Check what happens. You can use docker ps on the current node, and on another node.

In order to help visualize the state of the Swarm cluster you can use the visualizer companion of Swarm. On the master node run the following:

`#` **`docker run -it -d -p 8080:8080 -v /var/run/docker.sock:/var/run/docker.sock manomarks/visualizer`**

or using the service notion:

`#` **`docker service create --name=viz --publish=8080:8080/tcp --constraint=node.role==manager --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock dockersamples/visualizer`**

And then connect your browser to it on port 8080. You should see something similar to the below image:
![Swarm Visualizer](/Docker/img/visualizer.png)

Here you can experiment on meshing, connecting to any node, should send you to the required application.

Now let's deploy our application on our cluster. With recent versions of docker-compose, there is the new notion of stack to orchestrate services. Adapt your docker-compose to use it following the below model:
```
version: '3'
services:
  web:
    image: owncloud_web   # use previously generated image (used to be build: .)
    volumes:
      - /data/owncloud:/data/owncloud
      - /data/config:/var/www/html/owncloud/config
    ports:
      - "8000:80"
    networks:
      - oclan
    depends_on:
      - db
  db:
    image: mariadb
    ports:
      - "3306:3306"
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=owncloud
      - MYSQL_USER=owncloud
      - MYSQL_PASSWORD=owncloudpwd
    volumes:
      - /data/db:/var/lib/mysql
    networks:
      - oclan
networks:
  oclan:
    driver: overlay      # use overlay network
```

Note : The overlay network is a network that will use VXLAN technology to create a private network between hosts, which could be in the diffrent subnets.

Note 2 : behind the scene the init phase of swarm did a lot of complex things, VXLAN, security (everything is on top of TLS), meshing, load balancing. Also note that load balancing on physical nodes must be achieved by an external mechanism. Really nice job !

Now start your stack:

`#` **`docker stack deploy -c docker-compose-v3.yml oc`**
```
Ignoring unsupported options: build, links

Creating service oc_web
Creating service oc_db
```

`#` **`docker service ls`**
```
ID                  NAME                MODE                REPLICAS            IMAGE                 PORTS
dm2j7n185u53        oc_db               replicated          1/1                 mariadb:latest
g26e0mhakd6x        oc_web              replicated          1/1                 owncloud_web:latest   *:80->80/tcp
v12wk2jruwhl        relaxed_morse       replicated          10/10               alpine:latest
```

You may have some problems with this. Try to understand what happens and solve your issues. How many replicas are working ? Where are the images to use ? Which node can use them ?
Hint: use the command `docker stack services oc` to help diagnose. And as usual talk to your instructor !

So you will need to use a private registry here to help solving that issue.

We have deployed a Docker registry for you, available from a URL that will be provided by the instructor.
(If you use the internal HPE Lab, then try lab7-2.labossi.hpintelco.org:5500 - If you want to create your own, use our scripts at https://github.com/bcornec/Labs/tree/master/Docker/registry)

You need to add the CA public certificate made on the registry to trust it.
Download the CA from the registry web site:

### CentOS 7

`#` **`curl -L http://lab7-2.labossi.hpintelco.org/ca.crt > /etc/pki/ca-trust/source/anchors/ca-registry.crt`**

`#` **`update-ca-trust`**

`#` **`systemctl restart docker`**

### Ubuntu/Debian

`#` **`curl -L http://lab7-2.labossi.hpintelco.org/ca.crt > /usr/local/share/ca-certificates/ca-registry.crt`**

`#` **`update-ca-certificates`**

`#` **`service docker restart`**

Check that the registry runs as expected:
`#` **`curl -L https://<my-registry-fqdn>:5500/v2`**
`{}`

Of course, each node needs to be configured identically.

In order to share the image between the nodes, you need to push it to this new
registry, by using the appropriate tag. For example, you may use a command similar to

`#` **`docker tag owncloud_web:latest ${DOMAIN_NAME}:5500/owncloud_web`**

And then you can push that image into our registry so it's available to other engines to use.

`#` **`docker push ${DOMAIN_NAME}:5500/owncloud_web`**

Do the same with the mariadb service that you create afterwards following the same approach.
Look at your stack status. Is everything working fine or not ? What happens if you kill the httpd process ? the mysql process ? Explain what is happening.

Now for the storage it's more difficult as the volumes you want to mount should be, as the images previously, available on all engines so each container created on it can use these data. One way to solve this for the mariadb image is to use an NFS exported directory from your first node.
Let's configure NFS on the first machine (10.11.51.136 in my case):

`#` **`yum install -y nfs-utils`** # CentOS7

or

`#` **`apt-get install -y nfs-server`** # Ubuntu before 18.04

or

`#` **`apt-get install -y nfs-kernel-server`** # Ubuntu 18.04

Edit the exports file so it looks like:

`#` **`cat /etc/exports`**
```
/data/db        *.labossi.hpintelco.org(rw,no_root_squash,async,insecure,no_subtree_check)
/data/owncloud  *.labossi.hpintelco.org(rw,no_root_squash,async,insecure,no_subtree_check)
/data/config    *.labossi.hpintelco.org(rw,no_root_squash,async,insecure,no_subtree_check)
```
`#` **`exportfs -a`**

`#` **`systemctl start nfs`** # Ubuntu before 18.04

or 

`#` **`systemctl start nfs-kernel-server`**

Install on other nodes nfs client and check that your NFS setup is correct.

Hint:

`#` **`yum install -y nfs-utils`**

`#` **`systemctl start rpc-statd`** # CentOS7

or

`#` **`apt-get install -y nfs-common`**

`#` **`service rpc.statd start`** # Ubuntu

Now you can create a Docker volume that will be used by the containers launched with a service, by amending your docker-compose file which should now look like this:

```
version: '3'
services:
  web:
    build: .
    image: lab7-2.labossi.hpintelco.org:5500/owncloud_web
    volumes:
      - /data/owncloud:/data/owncloud
      - /data/config:/var/www/html/owncloud/config
    ports:
      - "8000:80"
    networks:
      - oclan
  db:
    image: mariadb
    ports:
      - "3306:3306"        # note that this port is exposed for the following part
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=owncloud
      - MYSQL_USER=owncloud
      - MYSQL_PASSWORD=owncloudpwd
    volumes:
      - dbvol:/var/lib/mysql
    networks:
      - oclan
networks:
    oclan:
        driver: overlay

volumes:
  dbvol:
    driver: local
    driver_opts:
      type: nfs
      o: addr=10.11.51.136,rw,nfsvers=4.1    # this is required to have locks as nfs V3 does not support lock required by mariadb
      device: ":/data/db"
```

Restart your stack:
`#` **`docker stack rm oc`**

`#` **`docker stack deploy -c docker-compose-v3.yml oc`**

Check they have now been created with:
`#` **`docker volume ls`**

BTW, you can see that Docker already transparently created many more volumes for you.
Note that you have to do it on all the engines of your Swarm cluster for this method to work.

Is that now working as expected ? If you use Docker 17.03+ you should have the docker service logs command now to help you diagnose your issue. If not, then tip is to use docker service ps <svc_id> to find on which host runs the service and then docker exec/logs on that host e.g. Also think to the /var/log/messages log file on your host.

Can you have access to the database with the mysql command from your host (install the command if you need it) ? Check that the volume is mounted correctly in the container. Check that you can reach the mysql daemon from any host in the cluster. For mysql to work correctly using an NFS exported directory for its files, you will need to have the rpc.statd daemon running on all nodes of your cluster.

Create a temporary table in the owncloud database to check and then relaunch the service to verify the persistency of the DB.
MariaDB hint:

`#` **`mysql -h <one node> -uowncloud -powncloudpwd`**

`MariaDB [(none)]>` **`use owncloud;`**

`MariaDB [(owncloud)]>` **`create table toto (id int);`**

`MariaDB [(owncloud)]>` **`show tables;`**

`MariaDB [(owncloud)]>` **`quit;`**

Once all this is solved, you can try dealing with the web frontend. Adopt a similar approach (NFS volume and service). Check that the communication between owncloud and the DB works fine.

You may be affected as myself by remaining bugs with previous versions of docker, such as https://github.com/docker/docker/issues/20486 or https://github.com/docker/docker/issues/25981, especially mixing tests with docker-compose and swarm. For me, the only way to turn around them was to reboot the full cluster completely.

Observe what happens when you restart a Docker service on a node hosting one of the 2 services.

We can scale out such a stateful application (while less interesting than a cloud native one) with many owncloud instances to support many users and spread the load across the Swarm cluster.

PLEASE, stop your services to avoid ports conflicts with the next part.

`#` **`docker stack rm oc`**

Now we'll see the adequation of Docker Swarm and Cloud Native applications.

# Deploy a cloud native application.

Let's explain first the application and its goal.

## Objectives

In this section, we will create a promotional lottery for an e-commerce site.
All the software components are provided, you'll "just" have to perform a partial containerization of the service.

As the setup takes some time, we'll start with the instructions and then you'll have time to read the explanations.

First have access to the application we developed for this.

`#` **`yum install -y git`**

`#` **`git clone https://github.com/bcornec/cloud_native_app.git`**

`#` **`cd cloud_native_app`**

As you can see in the cloud_native_app directory created, the same application can be used for a Docker or an OpenStack usage (or combining them).
The application is still a WIP/Demo, so don't worry with all the additional files and directories for now. Upstream is at https://github.com/uggla/openstack_lab.git alongside its documentation.

We need first to run the application locally using the compose file, in order to create all the Docker images and to upload them into the registry.

`#` **`./docker_services.sh`**

Drink a coffee, it's well deserved at that point, the composition takes a bit of time. Or stay looking at it to observe closely the magic of Docker automation ;-)
Please start reading the following explanations in or to understand what we're building for you here.

A customer of a big e-commerce site receives a promotional email with a link to win a prize if they are lucky.
The application detects whether the player already played or not, and whether he won already or not.
Each status is kept with the date when it was performed. The application provides a button allowing the customer to play, in case he didn't already, and the result of the computation which happens behind the scene is given back to the customer: it is the nature of the article he has won, and the corresponding image is displayed in the interface. Mails are sent to admins when a winner is found.

That application is made of one Web page with 5 parts/micro-services: I, S, B, W and P:
  - I(dentification) service: receives http request from customer (link with customer ID) and look for it into the DB.
  - S(tatus) service: detect whether customer already played or not, status stored in the DB. It is using a messages bus to buffer requests.
  - B(utton) service: button widget allowing the customer to play. Only when not already done.
  - W(orker) service that computes whether the customer won or not (slow service on purpose with a REST API interface), called by B. If won, post an image representing what has been won into an object store with customer ID. Then post by e-mail via an external provider a message to admins (using a messages bus). Button is gray if the customer has already played. W and the DB are on a separate private network.
  - P(icture) service: Look into the object store with customer ID to display the image of the customer's prize, empty if no image.

Each part of the web page is implemented as a micro-service. So the application supports nicely the death of any one of the 5 micro-services. The page is still displayed anyway, printing N/A when a micro-service is unavailable. In case of insufficient resources (as with the slow W micro-service), we will look at how to scale that application.

Please have a look at the `docker_services.sh` script and adapt what needs to be changed for your environment at the start in case of issues.

At the end of the script you should get a list of services running similar to the one below:
```
ID            NAME         REPLICAS  IMAGE                                                 COMMAND
1empjc9o6wwu  w            1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_w
1z53fru1vjr6  i            1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_i
3gasrkzgpp0w  b            1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_b
3sc3qexaixkl  redis        1/1       redis
4c5i32juwnyh  myownsvc     1/1       lab7-2.labossi.hpintelco.org:5500/owncloud_web
5yl1168mm6h4  w2           1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_w2
6leldkqf1zth  ping         global    alpine                                                ping 8.8.8.8
79jwqr43zyt2  web          1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_web
7hygz6g0lbyq  db           1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_db
9i4ogenk03ax  rabbit       1/1       rabbitmq:3-management
ag12vg6ts417  tiny_curran  10/10     alpine                                                ping 8.8.8.8
ajcrqc6nykn8  s            1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_s
cn81a9a5j8yi  w1           1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_w1
e6c6ypgcxdy2  p            1/1       lab7-2.labossi.hpintelco.org:5500/cloudnativeapp_p
```

In order to use the application you'll now have to connect to your system hosting the web application (in our case http://c6.labossi.hpintelco.org/)

![cna](Docker/img/cna.png)

You should see a message in your browser saying:
```
Please provide a user id !
```

So now to use the application, you have to provide the id of the user who is playing to see his prize.
Browse http://c6.labossi.hpintelco.org/index.html?id=1

Check the availability of the application by restarting a docker daemon on a host running one of the containers the application is using.
Check the micro-service behavior by stopping the 'i' micro-service, and then the 'p' micro-service. Reload the Web page each time to see what happens.

Try to make more connections. What is the problem encountered.
Which micro-service is causing the issue.
Scale that micro-service to solve the problem.

This is the end of this lab for now, we hope you enjoyed it.

Github issues and pull requests to improve this lab are welcome.
